In [1]:
import numpy as np
import pandas as pd
import pymystem3
import razdel
import json
from sklearn.metrics.pairwise import cosine_similarity

from collections import defaultdict

In [2]:
import re

SPLIT_RGX = re.compile(u'[A-Za-zА-Яа-я0-9]+', re.UNICODE)

def split(string):
    words = re.findall(SPLIT_RGX, string)
    return words

In [3]:
stem = pymystem3.Mystem()

In [4]:
def get_docs(file_name):
#     init_docs = []
    docs = []
    with open(file_name, 'r') as f:
        for i, line in enumerate(f):
            t = razdel.sentenize(line)
            for e in t:
                d = {}
                d['q_idx'] = i
                d['init'] = e.text.strip()
#                 init_docs.append((i, e.text.strip()))
                lemmas = stem.lemmatize(e.text)
                doc = ''.join(lemmas)
                words = split(doc)
                d['lemmas'] = words
#                 docs.append((i, words))
                docs.append(d)
    return docs

In [5]:
def get_vocab(docs):
    lemmas = set()
    for d in docs:
        for lemma in d['lemmas']:
            lemmas.add(lemma)
            
    lemma2id = {lemma: i for i, lemma in enumerate(lemmas)}
    
    return lemma2id

In [6]:
def get_tf_idf(lemma2id, docs):
    vocab_len = len(lemma2id)
    
    df = np.zeros(vocab_len)
    tfs = []
    for d in docs:
        tfs.append(np.zeros(vocab_len))
        for lemma in set(d['lemmas']):
            if lemma not in lemma2id:
                continue
            df[lemma2id[lemma]] += 1
        for lemma in d['lemmas']:
            if lemma not in lemma2id:
                continue
            tfs[-1][lemma2id[lemma]] += 1
    
    idf = np.log(len(docs)/df)
    tfs = np.vstack(tfs)
    
    return tfs, idf

In [20]:
def model_01(tfs, idf):
    t = tfs * idf
    t = t / np.linalg.norm(t, axis=1).reshape(-1, 1)
    return t

def model_02(tfs, idf):
    t = ((0.5 + 0.5 * (tfs / tfs.max(axis=1).reshape(-1, 1))) * (tfs > 0) )* idf
    t = t / np.linalg.norm(t, axis=1).reshape(-1, 1)
    return t

def prop_model(docs_tfs, qs_tfs, alpha):
    glob_props = docs_tfs.sum(axis=0)
    glob_props /= glob_props.sum()
    scores = []
    loc_props = docs_tfs / docs_tfs.sum(axis=1).reshape(-1, 1)
    for i in range(qs_tfs.shape[0]):
        scores.append((
            np.log(0.000001 +((1-alpha)*glob_props + alpha*loc_props)) *
            (qs_tfs[i] > 0).astype(int)
         ).sum(axis=1))
    scores = np.array(scores)
    return scores.T

In [8]:
with open('rated_docs.json') as f:
    docs = json.load(f)
for i in range(len(docs)):
    r = int(docs[i]['rating'])
    rating = [0, 0, 0]
    rating[docs[i]['q_idx']] = r
    docs[i]['rating'] = rating

In [9]:
scores = []
for d in docs:
    scores.append(d['rating'])
scores = np.array(scores)

In [10]:
lemma2id = get_vocab(docs)
docs_tfs, idf = get_tf_idf(lemma2id, docs)
doc_m_1 = model_01(docs_tfs, idf)
doc_m_2 = model_02(docs_tfs, idf)

In [11]:
qs = get_docs('queries.txt')
qs_tfs, _ = get_tf_idf(lemma2id, qs)
qs_m_1 = model_01(qs_tfs, idf)
qs_m_2 = model_02(qs_tfs, idf)

/tmp/ipykernel_7976/1109202693.py:17: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log(len(docs)/df)


In [12]:
scores_m_1 = cosine_similarity(doc_m_1, qs_m_1)
scores_m_2 = cosine_similarity(doc_m_2, qs_m_2)

In [29]:
scores_prop_05 = prop_model(docs_tfs, qs_tfs, 0.5)
scores_prop_09 = prop_model(docs_tfs, qs_tfs, 0.9)

In [14]:
def dcg(scores, model_scores, k=10):
    return (scores[model_scores.argsort()[::-1]][:k] / np.log2(np.arange(k) + 2)).sum()

def ndcg(scores, model_scores, k=10):
    dcg_ = dcg(scores, model_scores, k)
    idcg_ = dcg(scores, scores, k)
    return dcg_ / idcg_

def make_report(docs, qs, scores, model_scores, top_k=10, dcg_k=10):
    for i in range(len(qs)):
        print(f'Query {i+1}: {qs[i]["init"]}\n')
        print(f'NDCG: {ndcg(scores[:, i], model_scores[:, i], dcg_k)}')
        print(f'Top {top_k}:\n')
        for k, idx in enumerate(model_scores[:, i].argsort()[::-1][:top_k]):
            print(f'{k+1} (true: {round(scores[:, i][idx], 5)}) (model: {round(model_scores[:, i][idx], 5)}): {docs[idx]["init"]}\n')
        print(20*'#' + '\n')

In [35]:
make_report(docs, qs, scores, scores_prop_09)

Query 1: Ракетный корпус княжества Майсур в 1780 году нанёс британской армии одно из самых тяжёлых поражений.

NDCG: 0.8780715435102542
Top 10:

1 (true: 2) (model: -62.67476): Во время Второй англо-майсурской войны в битве при Поллилуреruen (1780 год) разрушительный обстрел ракетного корпуса Типу поджёг склады боеприпасов Ост-Индской компании, нанеся британской армии одно из самых тяжёлых поражений в Индии.

2 (true: 1) (model: -106.95115): Майсур стал первым индусским туземным княжеством в Британской Индии.

3 (true: 1) (model: -109.23699): Майсурские ракеты - ракетное оружие, разработанное и успешно применявшееся южноиндийским мусульманским княжеством Майсур в XVIII веке в англо-майсурских войнах.

4 (true: 1) (model: -113.08699): Фактический правитель Майсура XVIII века Хайдер Али разработал первые прототипы более прочных ракет, наполненных взрывчаткой, с корпусом из металла (до этого он изготавливался из картона).

5 (true: 0) (model: -113.58624): Княжество Майсур.

6 (true: 1) (m